<a href="https://colab.research.google.com/github/GKCSV/Resilia_ProjetoFinal_M6/blob/GK/Analise_de_despesas_publicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. ANÁLISE DE DESPESAS PÚBLICAS GERADAS POR DEPUTADOS ESTADUAIS NOS ANOS DE 2019, 2020 E 2021#

##1.1 Inicialização do ambiente e importação dos dados:##

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from matplotlib.ticker import StrMethodFormatter
from sklearn.preprocessing import MinMaxScaler

In [2]:
despesas2019 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2019.csv', sep = ';')
despesas2020 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2020.csv', sep = ';')
despesas2021 = pd.read_csv('https://raw.githubusercontent.com/GKCSV/Resilia_ProjetoFinal_M6/main/Ano-2021.csv', sep = ';')

##1.2 Formatação do dataset:##

###1.2.1 Mesclando dataset's para um único dataset:###

In [3]:
despesas1920 = pd.merge(despesas2019, despesas2020, how='outer')
despesas_gerais = pd.merge(despesas1920, despesas2021, how='outer')

###1.2.2 Exclusão de colunas não-utilizadas através do comando .drop()###

In [4]:
excluir_colunas = despesas_gerais.drop(columns=['nuCarteiraParlamentar', 'nuLegislatura', 'numParcela', 'datEmissao', 'codLegislatura', 'numEspecificacaoSubCota', 'txtNumero', 'indTipoDocumento', 'numLote', 'ideDocumento', 'urlDocumento'])
excluir_colunas

,txNomeParlamentar,cpf,ideCadastro,sgUF,sgPartido,numSubCota,txtDescricao,txtDescricaoEspecificacao,txtFornecedor,txtCNPJCPF,vlrDocumento,vlrGlosa,vlrLiquido,numMes,numAno,txtPassageiro,txtTrecho,numRessarcimento,vlrRestituicao,nuDeputadoId
0,LID.GOV-CD,NaN,NaN,NaN,NaN,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,CRISTIANE FERREIRA EPP,037.139.030/0010-8,111.30,0.0,111.30,1,2019,NaN,NaN,NaN,NaN,2812
1,LID.GOV-CD,NaN,NaN,NaN,NaN,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,CRISTIANE FERREIRA EPP,037.139.030/0010-8,117.19,0.0,117.19,2,2019,NaN,NaN,NaN,NaN,2812
2,LID.GOV-CD,NaN,NaN,NaN,NaN,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,CRISTIANE FERREIRA EPP,037.139.030/0010-8,120.15,0.0,120.15,2,2019,NaN,NaN,NaN,NaN,2812
3,LID.GOV-CD,NaN,NaN,NaN,NaN,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,CRISTIANE FERREIRA EPP,037.139.030/0010-8,151.18,0.0,151.18,3,2019,NaN,NaN,NaN,NaN,2812
4,LID.GOV-CD,NaN,NaN,NaN,NaN,13,FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR,NaN,PANIFICADORA FERREIRA NETO LTDA - EPP,037.139.030/0010-8,119.90,0.0,119.90,5,2019,NaN,NaN,NaN,NaN,2812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555495,Henrique do Paraíso,3.470995e+10,217036.0,SP,REPUBLICANOS,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,NaN,FABIO FRERICHS TREVISAN PUBLICIDADE,151.600.920/0014-4,5000.00,0.0,5000.00,7,2021,NaN,NaN,NaN,NaN,3486
555496,Henrique do Paraíso,3.470995e+10,217036.0,SP,REPUBLICANOS,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,NaN,LAR COMUNICAÇÃO VISUAL,328.697.980/0010-7,7000.00,0.0,7000.00,8,2021,NaN,NaN,NaN,NaN,3486
555497,Henrique do Paraíso,3.470995e+10,217036.0,SP,REPUBLICANOS,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,NaN,LAR COMUNICAÇÃO VISUAL,328.697.980/0010-7,4000.00,0.0,4000.00,7,2021,NaN,NaN,NaN,NaN,3486
555498,Henrique do Paraíso,3.470995e+10,217036.0,SP,REPUBLICANOS,10,TELEFONIA,NaN,RAMAL,000.000.000/0000-6,1.10,0.0,1.10,7,2021,NaN,NaN,0.0,NaN,3486
